# A. All Needed Modules

In [ ]:
# !pip install torch
# !pip install pdfminer
# !pip install pytesseract
# !pip install tensorflow
# !pip install pdf2image
# !pip install transformers
# !pip install spacy
# !spacy download en_core_web_lg
# !pip install opencv-python
# !pip install seqeval
# !pip install evaluate
# !pip install huggingface_hub

# B. Account Login

In [1]:
# from huggingface_hub import notebook_login
# from huggingface_hub import upload_file
# from huggingface_hub import delete_file
# from huggingface_hub import Repository
# notebook_login()

# C. Import Modules

In [3]:
#PDF Miner (PDF Scraping)
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import HTMLConverter, TextConverter, XMLConverter
from io import StringIO
from pdfminer.pdfpage import PDFPage

#OCR
from pdf2image import convert_from_path
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'Tesseract-OCR\tesseract.exe'


#NLP Preprocessing
import nltk
import re
import regex
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


#NLP Processing TensorFLow
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

#NLP Processing Spacy
import spacy
# import spacy_transformers
# spc = spacy.load('en_core_web_trf')
spc = spacy.load('en_core_web_lg')
# spc.add_pipe('sentencizer')

#Model Training and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import torch

#Basic
import numpy as np
import pandas as pd
import pickle
import glob
import os
# import docker
import string
import matplotlib.pyplot as plt

pc = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~'

# D. All Functions for Text Preprocessing 1 (V2.0)

In [4]:
def start_end(b):
    c = b.lower()
    try:
        awal = [(m.start(0), m.end(0)) for m in re.finditer(r'1 introduction', c)]
        awal_ind = awal[-1][-1]
    except:
        try:
            awal = [(m.start(0), m.end(0)) for m in re.finditer(r'keywords', c)]
            awal_ind = awal[0][-1]
        except:
            awal_ind = 0

    akhir = [(m.start(0), m.end(0)) for m in re.finditer(r'credit authorship contribution statement', c)]
    if len(akhir) == 0:
        try:
            akhir = [(m.start(0), m.end(0)) for m in re.finditer(r'declaration of competing interest', c)]
            akhir_ind = akhir[-1][0]
        except:
            try:
                akhir = [(m.start(0), m.end(0)) for m in re.finditer(r'acknowledgments', c)]
                akhir_ind = akhir[-1][0]
            except:
                try:
                    akhir = [(m.start(0), m.end(0)) for m in re.finditer(r'references', c)]
                    akhir_ind = akhir[-1][0] 
                except:
                    akhir_ind = None
    else:
        akhir_ind = akhir[-1][0]
    
    return b[awal_ind:akhir_ind]

def preprocessing1_v2(text_pdfmine):
    #Punctuation of '-' because make a newline 
    layers = re.sub(r'(-\n?\s)','',text_pdfmine)
    
    #Remove spacing n dot '\n' & '•'
    if bool(regex.search(r'(?<=(\w\w))[\n•](?=(\w\w))',layers))==True:
        layers = re.sub(r'[\n•]', ' ', layers)
    else:
        layers = re.sub(r'[\n•]', '', layers)
        
    #Adjustment in Germany alphabetic
    layers = re.sub(r'(?<=[oua])(¨)','e',layers)
    layers = re.sub(r'ß','ss',layers)
    
    #Delete et al. + lower
    layers = regex.sub(r'et al.','with colleagues',layers)
    
    #Eliminate fig and table
    layers = regex.sub(r'(\b[Ff]igs*\.*\b)', 'Figure', layers)
    layers = regex.sub(r'(?<=[Tt]ables*)[\s+]*[\d+]*\.*', ' ',layers)
    
    #Eliminate email
    layers = regex.sub(r'\b[\w-.]+?@\w+?.\w+[\w\.\-]+\b',' ',layers)

    #Eliminate website
    layers = regex.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w\s_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',' ',layers)
    
    #Adjusment in Over-Spacing
    layers = regex.sub(r'(\s)(?=\s+)','',layers)
    
    #Adjutsment Formation in Geology
    layers = regex.sub(r'[Ff]m.','Formation',layers)
    
    #Splitted into sentences by Spacy
    spc_text = spc(layers)

    #Preparation
    prepro2=[]
    
    #List of Sentences
    for i,j in enumerate(spc_text.sents):
        if bool(regex.search(r'^,',j.text)) == False:
            prepro2.append(j.text)
        else:
            prepro2[-1] = prepro2[-1] + j.text
    
    #Pattern for remove bracket
    pattern1 = regex.compile(r'{}[^{}]+{}'.format('\(','\)','\)'))
    
    #Pattern for remove bracket
    pattern2 = regex.compile(r'{}[^{}]+{}'.format('\[','\]','\]'))
    
    #Pattern for unused space first n last
    pattern3 = regex.compile(r'((^\s+)(?=.)|(?<=.)(\s+$))')

    #Pattern for Apply over-space
    pattern4 = regex.compile(r'(\s)(?=\s+)')

    #Pattern for cleaning
    for i,j in enumerate(prepro2):
        
        #Remove Bracket
        layer = regex.sub(pattern1, ' ', j)
        
        #Remove Bracket
        layer = regex.sub(pattern2, ' ', layer)
        
        #With Removing unused space first n las
        layer = regex.sub(pattern3,'',layer)
        
        #With Removing over-space
        prepro2[i] = regex.sub(pattern4,'',layer)
        
        #If no dot in the last
        try:
            if prepro2[i][-1] != '.':
                prepro2[i] = prepro2[i] + '.'
            else:
                pass
        except:
            pass

    #Final filtering for less than three words sentence
    d = lambda y : None if len(y.split(' '))<=4 else y
    prepro2 = list(filter(d, prepro2))
    

    return prepro2

# E. Self-Supervised - Rule-Based Auto Labeling

## E.1 Load Dictionary of Words for all Entities

In [5]:
#Auto Read entities
dict_entities={}
path='list_entities - V4-tes/'
for filename in glob.glob(os.path.join(path, '*.txt')):
    gist_file = open(filename, "r", encoding = "ISO-8859-1")
    try:
        content = gist_file.read().lower()
        dict_entities[re.search(r'(\w+).txt$',filename).group(1)] = list(filter(None, content.split("\n")))
    finally:
        gist_file.close()

In [6]:
dict_entities

{'GeoLoc': ['galicia - tras - os - montes zone',
  "sana ' a - amran volcanic field",
  '06 - 11 - 051 - 26w4',
  '15 - 23 - 036 - 10w5',
  '10 - 31 - 37 - 9w5',
  'st - joseph - de - beauce',
  'st - marc - des - carrières',
  'yacyreta p . p . reservoir',
  'gronau ( eichel - berg )',
  'united saint tes minor outlying islands',
  "jabal an nabi shu ' ayb",
  'masjid - i - sulaiman ir',
  'saint joseph - de - beauce',
  'saint marc - des - carrieres',
  'st . marc des carrieres',
  'democratic republic of the congo',
  'united states minor outlying islands',
  'referred to as gondwana island',
  'saint inbrueche e bad orb',
  'alter saint inbruch am hirschberg',
  'saint inbrueche e bad orb',
  'cerro bolsón de los cerillos',
  'mid - atlantic ocean ridge',
  'ghanzi - chobe fold belt',
  'caribbean - columbian oceanic plateau',
  "sana ' a and amran",
  'uetikon a . s .',
  'oetwil a . l .',
  'yacyreta p . p .',
  'sikhote - alin - ridge',
  'lhd7 . 8 - 1568',
  'colmar - sud - 1',

## E.2 Entities Encoders Mapping for Training

In [7]:
ent_map = {'B-GeoTime': 1,
 'O': 0,
 'I-GeoTime': 2,
 'B-GeoLoc': 3,
 'I-GeoLoc': 4,
 'B-GeoMeth': 5,
 'I-GeoMeth': 6,
 'B-GeoPetro': 7,
 'I-GeoPetro': 8, 'None': -100}

In [8]:
reverse_ent_map = {i:j for j,i in ent_map.items()}
reverse_ent_map

{1: 'B-GeoTime',
 0: 'O',
 2: 'I-GeoTime',
 3: 'B-GeoLoc',
 4: 'I-GeoLoc',
 5: 'B-GeoMeth',
 6: 'I-GeoMeth',
 7: 'B-GeoPetro',
 8: 'I-GeoPetro',
 -100: 'None'}

## E.3 Text Data Preparation

### E.3.1 Text Data Preparation: Data from BGS NER

In [9]:
def read_predefined_token(file_str):
    file_str = 'bgs.3class.geo-all-data.txt' 
    pd.read_fwf(file_str,encoding="latin-1",header=None)

    import re
    prepro2 = []
    with open(file_str, 'r') as f:
        lines = f.readlines()
        for i in lines:
            prepro2.append((re.sub(r'(\t|\n)',' ',i)).split(' '))

    for i in prepro2:
        i.remove('')

    dictionary = {'LEXICON':'GeoPetro', 'CHRONOSTRAT':'GeoTime', 'BIOZONE':'GeoPetro', 'O':'O'}

    sent = []
    temp_sent = []
    code = []
    coo = []
    temp_code = []
    for i in prepro2:
        if len(i)>1:
            temp_sent.append(i[0])
            if i[1] != 'O':
                temp_code.append("B-" +dictionary[i[1]])
            else:
                temp_code.append(dictionary[i[1]])

            try:
                if temp_code[-1] == temp_code[-2] and temp_code[-1] !='O':
                    temp_code[-1] = "I-" + temp_code[-1][2:]

                elif temp_code[-2] == "I-" + temp_code[-1][2:] and temp_code[-1] !='O':
                    temp_code[-1] = "I-" + temp_code[-1][2:]

                else:
                    pass
            except:
                pass
        else:
            sent.append(temp_sent)
            code.append(temp_code)
            temp_sent = []
            temp_code = []

    for n,m in enumerate(sent):
        i = ' '.join(m).lower()

        for j,k in enumerate(list(dict_entities.values())):
            pat = r",*(\b(?:{})e*s*)\b".format("|".join(k))
            sear = re.findall(pat,i)

            try:
                if type(sear[0]) == tuple:
                    sear = list(filter(None, [d for d in sear[0]]))
                else:
                    pass
            except:
                pass

            if bool(sear) == True:
                m = [i.lower() for i in m]
                for l in sear:
                    if len(l.split(' ')) > 1:

                        try:
                            first = l.split(' ')[0]

                            ind_change = m.index(first)

                            if code[n][ind_change] == 'O':

                                code[n][ind_change] = 'B-' + list(dict_entities.keys())[j]

                                for b in range(1,len(l.split(' '))):
                                    code[n][ind_change+b] = 'I-'+ list(dict_entities.keys())[j]

                            else:
                                pass

                        except:
                            pass

                    elif len(l.split(' ')) == 1:
                        try:
                            if code[n][m.index(l)] == 'O':
                                code[n][m.index(l)] = 'B-' + list(dict_entities.keys())[j]
                            else:
                                pass
                        except:
                            pass
                    else:
                        pass
            else:
                pass

    for i, j in enumerate(sent[:10]):
        print(list(zip(j,code[i])))
        print(' ')
        
    return(sent, code)

In [10]:
## Conduct the processing
# read_predefined_token('bgs.3class.geo-all-data.tx')

### E.3.2 Text Data Preparation: Data from Webscraping

In [11]:
def prepro1(path):
    cc = []
    
    #Read All Text Data (Webscraped, PDF-Scraped, BGS NER)
    for filename in glob.glob(os.path.join(path, '*.txt')):
        with open(filename, "r", encoding = 'utf-8') as f:
            lines = f.read()
        c = lines.split('\n----END----\n----NEW----\n')
        
        #Apply the Preprocessing 1 pipeline
        for i in c:   
            cc.append(preprocessing1_v2(regex.sub(r'\n',' ',i)))
   
    #saved it as file.
    with open('preprocessed_text_full.txt', 'w', encoding='utf-8') as f:
        for i in cc:
            for j in i:
                f.write(j)
                f.write('\n')
        f.close()

In [ ]:
# prepro1('txt_data/')

In [12]:
def prepro1_single(path):
    #Read particular Text Data (from Webscraped, PDF-Scraped, BGS NER)
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.read()
    c = lines.split('\n----END----\n----NEW----\n')
    cc = []
    
    #Apply the Preprocessing 1 pipeline
    for i in c:
        cc.append(preprocessing_v2(regex.sub(r'\n',' ',i)))
    
    #saved it as file.
    with open('preprocessed_text_full.txt', 'w', encoding='utf-8') as f:
        for i in cc:
            for j in i:
                f.write(j)
                f.write('\n')
        f.close()

In [ ]:
# prepro1_single('Text Data for NLP/tes.txt')

In [13]:
def read_direct(path):
    #Directly read the Preprocessed 1 text data
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.read()
    c = lines.split('\n')
    cc = []
    for i in c:
        cc.append(i)
    return cc

In [ ]:
# preprocessed1 = read_direct('preprocessed_text_full.txt')

## E.4 Define the Fine-Tuned Model for TRANSFER LEARNING: GeoBERT

In [14]:
# Import Tokenizer architecture from GeoBERT
from transformers import AutoTokenizer

model_checkpoint = "botryan96/GeoBERT"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## E.5 All Functions for Self Supervised - Rule Based Auto Labeling

In [15]:
def get_training_corpus():
    for start_idx in df.sent.values.tolist():
        yield start_idx
        
#Ready for NLP NER V8.0 (BIO Encoding V2)
def bio_encoding(sent):
    #Tokenization

    #Preparation
    r = []
    list_word = []
    a4_token = []
    b4_token = sent.split(' ')
    for b in b4_token:
        a4_token.append(tokenizer(b).tokens()[1:-1])
        r.append(tokenizer(b).tokens()[1:-1])

    #Check entities in the sentence.
    for j,k in enumerate(list(dict_entities.values())):
        pat = r",*(\b(?:{})(?:e*s*))\b".format("|".join(k))
        se = sent.lower()
        sear = re.findall(pat,se)


        #Adjust Tuple into List format from regex.findall result 
        try:
            if type(sear[0]) == tuple:
                c = []
                for i,m in enumerate(sear):
                    c.append(list(filter(None, [d for d in m])))
                c = sum(c,[])
                sear=list(c)
            else:
                pass
        except:
            pass

        #Grab all entities for BIO Encoding
        if bool(re.search(pat,se)) == True:
            for l in adjust_order(sear):
                list_word.append(l)
                try:
                    length_l = len(l.split(' '))

                    #Check the position of entities and change it
                    ind_pos = regex.sub(r'{}'.format(l),'O',se).split(' ').index('O')
                    for i in range(length_l):
                        try:
                            num = b4_token.index(l.split(' ')[0], ind_pos, ind_pos+1)
                        except:
                            pass
                        b4_token[ind_pos + i] = 'Z'

                    se = ' '.join(b4_token)
                    
                    #Grab entities with more than one words
                    if length_l > 1:
                        k=0
                        for d,h in enumerate(a4_token[num]): 
                            if k == 0:
                                a4_token[num][d] = 'B-' + list(dict_entities.keys())[j]
                                k = k + 1
                            elif k != 0:
                                a4_token[num][d] = 'I-' + list(dict_entities.keys())[j]
                            else:
                                pass

                        h = 0
                        for k in l.split(' ')[1:]:
                            h = h + 1
                            for s,a in enumerate(a4_token[num + h]):
                                if a4_token[num + h][s]:
                                    a4_token[num + h][s] = 'I-' + list(dict_entities.keys())[j]
                                else:
                                    pass

                    #Grab entities with one word
                    elif length_l == 1:
                        k=0
                        for d,h in enumerate(a4_token[ind_pos]): 
                            if k == 0:
                                a4_token[num][d] = 'B-' + list(dict_entities.keys())[j]
                                k = k + 1
                            elif k != 0:
                                a4_token[num][d] = 'I-' + list(dict_entities.keys())[j]
                            else:
                                pass
                    
                    #Normalize the Sentence
                    sent = " ".join(sum(a4_token,[]))        
                except:
                    pass
    
    pad = [None] 
    final = pad + sum(a4_token,[]) + pad
    
    list_map = list(ent_map.keys())
    for i,j in enumerate(final):
        if j in list(list_map) or j == None:
            pass
        else:
            final[i] = 'O'
            
    return final, list(zip(r,a4_token)), list_word


def ner_encoded(input_ids):
    return [ent_map[str(i)] for i in input_ids]

def aturi(c):
    d = []
    d_num = []
    for i,j in enumerate(c):
        if type(j) == str:
            d.append(j)
            d_num.append(i)
        elif type(j) == list:
            for k in j:
                d.append(k)
                d_num.append(i)
        else:
            pass
    return(d, d_num)

def bio_encoding_predefined(m_token, m_code):
    #Define the predefined Tokens Result
    b4_token = list(m_token)

    #Define the current Tokens Result
    prep = tokenizer(' '.join(m_token))

    a4_token = []
    for i in set(prep.word_ids()[1:-1]):
        start, end = prep.word_to_chars(i)
        a4_token.append(' '.join(b4_token)[start:end].lower())

    beda = list(set([i for i in b4_token if i.lower() not in a4_token]))

    #Solution for different Tokenization results
    #If there is different in Tokenization result
    if len(beda) != 0:
        dd_array = np.array(b4_token)
        for i in beda:
            ind = np.where(dd_array==i)[0]
            for k in ind:
                b4_token[k] = tokenizer(b4_token[k]).tokens()[1:-1]

        #Adjusment for Pre-defined Tokenization
        dic_1={}
        for i,j in enumerate(m_code):
            dic_1[i] = j

        aturi_op = aturi(b4_token)

        input_ids = [dic_1[j] for j in aturi_op[1]]

        prep_at = np.array(aturi_op[1])

        for i in list(set(aturi_op[1])):
            res = np.where(prep_at == i)
            if len(res[0]) > 1:
                for i in res[0][1:]:
                    input_ids[i] = re.sub(r'B-','I-',input_ids[i])
            else:
                pass

        #Adjusment for Current Tokenization
        dic_2={}
        for i,j in enumerate(input_ids):
            dic_2[i] = j

        input_ids = [dic_2[j] if j!= None else None for j in prep.word_ids()]

        prep_at = np.array(prep.word_ids())

        for i in list(set(prep.word_ids()[1:-1])):
            res = np.where(prep_at == i)
            if len(res[0]) > 1:
                for i in res[0][1:]:
                    input_ids[i] = re.sub(r'B-','I-',input_ids[i])
            else:
                pass

    #If there is no different in Tokenization result
    else:
        #Adjusment for Current Tokenization
        dic_1={}
        for i,j in enumerate(m_code):
            dic_1[i] = j

        input_ids = [dic_1[j] if j!= None else None for j in prep.word_ids()]

        prep_at = np.array(prep.word_ids())

        for i in list(set(prep.word_ids()[1:-1])):
            res = np.where(prep_at == i)
            if len(res[0]) > 1:
                for i in res[0][1:]:
                    input_ids[i] = re.sub(r'B-','I-',input_ids[i])
            else:
                pass

    print('New Tokenization Result:')
    print('-------')
    print(list(zip(prep.tokens(),input_ids)))
    
    return prep, input_ids

def adjustment(data,sent):
    if len(set(data)) == 2:
        data = tokenizer(sent).word_ids()
    else:
        pass
    return data

def adjust_order(r):
    content_com = []
    max_space = max([len(re.findall(r"\s",i)) for i in r])

    for i in range(0,max_space+1):
        locals()['space_'+str(i)] = []

    for i in r:
        s = re.findall(r"\s",i)
        locals()['space_'+str(len(s))].append(i)

    for i in range(max_space,-1,-1):
        content_com = content_com + locals()['space_'+str(i)]
    
    return content_com

def adjust_first(sent):
    #Sentence Treatment b4 tokenization
    pattern1 = regex.compile(r'[]!"$%&\'()’–*´·+,./:;=#@?[\\^_`{|}~-]+') #Spacing
    pattern2 = regex.compile(r'[^\w&\'\)\(,-./´·–’]') #only used punctuation
    pattern3 = regex.compile(r'(\s)(?=\s+)') #Over Spacing
    
    sent_clear = regex.sub(pattern1, r' \g<0> ', sent.lower()).strip()
    sent_clear = regex.sub(pattern2,' ',sent_clear)
    sent_clear = regex.sub(pattern3,'',sent_clear)
    
    return sent_clear

In [ ]:
sent = "upper carboniferous and lower permian sedimentary deposits deposited during upper carboniferous and lower permian"
bio_encoding(sent)

## E.5 Read & Prepare the Dataframe for Pre-Processing 2

In [16]:
#Read the preprocessed 1 text Data file
df_6 = pd.read_csv('partition_full/part_1_full_auto.csv')
df_6.drop('Unnamed: 0', inplace = True, axis=1)

#Reading it for every 20.000 sentences
with open('initiator.txt','r') as f:
    i = int(f.read())
awal = 10000*(i-1)*2 + 1
akhir=10000*(i)*2
with open('initiator.txt','w') as f:
    f.write(str(i+1))
print(i)

#Define it as Dataframe
df = df_6.copy()[awal:akhir]

In [ ]:
#Final Adjustment for Self-Supervised - Rule-Based Auto Labeling
dd = df.sent.values.tolist()
del df
new_one = []

result = sum(list(dict_entities.values()), [])
pat = r",*(\b(?:{})(?:e*s*))\b".format("|".join(result))

for j in dd:
    if bool(re.search(pat,j)) == False:
        pass
    else:
        new_one.append(j)
        
df = pd.DataFrame(new_one, columns=['sent'])
del dd
del new_one

In [17]:
df

,sent
20001,"int . pittsburgh coal con , f , 1996 , 13 , , ..."
20002,three types of coals varying in rank and moist...
20003,pretreatment of sub - bituminous coal by water...
20004,97 / 02502 the electrical and thermal characte...
20005,97 / 02503 excluded / included mineral matter ...
...,...
39995,ieee transactions on geoscience & remote sensi...
39996,inseason mapping of crop type with optical and...
39997,application of multi - temporal envisat asar d...
39998,"international journal of remote sensing , 31 ,..."


In [18]:
#Built a new Tokenizer with the GeoBERT architecture
old_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus(), 30000)

In [19]:
#Build Dataframe for Preprocessed 2 data
df_tok = df.copy()
d = df_tok['sent'].apply(lambda x: tokenizer(x))
df_tok['sent_tokens'] = d.apply(lambda x: x.tokens())
df_tok['word_ids'] = d.apply(lambda x: x.word_ids())
df_tok['input_ids'] = d.apply(lambda x: x.data['input_ids'])
df_tok['attention_mask'] = d.apply(lambda x: x.data['attention_mask'])
df_tok['token_type_ids'] = d.apply(lambda x: x.data['token_type_ids'])
df_tok['ner_tags'] = df_tok.apply(lambda x: bio_encoding(x['sent'])[0], axis=1)
df_tok['labels'] = df_tok['ner_tags'].apply(lambda x: ner_encoded(x))

In [ ]:
#Check the Data Preprocessed 2 Quality
def check_df(df_data):
    bb = df_data.labels.apply(lambda x: list(set(x))).values.tolist()

    try:
        d = bb.index([0,-100])
        print('Error in: ' + d)
        print(' ')

    except:
        print('All Data Good')

In [ ]:
# check_df(df_tok)

In [ ]:
# Splitting Data (Train, Test, and Validation) in Dataset Dictionary format
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

train_dataset, validation_dataset = train_test_split(df_tok[["attention_mask", "input_ids", "labels", "token_type_ids"]], test_size=0.30, random_state=42)
validation_dataset, test_dataset = train_test_split(validation_dataset, test_size=0.2, random_state=42)

train_dataset = Dataset.from_dict(train_dataset)
test_dataset = Dataset.from_dict(test_dataset)
validation_dataset =  Dataset.from_dict(validation_dataset)
my_dataset_dict = DatasetDict({"train":train_dataset,"test":test_dataset, "validation": validation_dataset})

# F. GeoBERT_NER Data Training

In [21]:
#Use the Data Collators scheme for Efficiency in Padding - Truncation 
from transformers import DataCollatorForTokenClassification
Tok
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer
)

In [22]:
batch = data_collator([my_dataset_dict["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    5,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    5,    6,    0, -100, -100, -100, -100, -100,
         -100],
        [-100,    0,    0,    0,    0,    0,    5,    0,    0,    0,    0,    0,
            0,    0,    5,    0,    0,    5,    0,    0,    0,    0,    0,    0,
         -100]])

In [23]:
#Apply Data Collators for data Training and Validation
tf_train_dataset = my_dataset_dict["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=5,
)

tf_eval_dataset = my_dataset_dict["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=5,
)

C:\Users\Admin\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:716: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [24]:
#Declare the Label scheme
id2label = {
 '0': 'O',   
 '1': 'B-GeoTime',
 '2': 'I-GeoTime',
 '3': 'B-GeoLoc',
 '4': 'I-GeoLoc',
 '5': 'B-GeoMeth',
 '6': 'I-GeoMeth',
 '7': 'B-GeoPetro',
 '8': 'I-GeoPetro'}

label2id = {
 'O': '0',
 'B-GeoTime': '1',
 'I-GeoTime': '2',
 'B-GeoLoc': '3',
 'I-GeoLoc': '4',
 'B-GeoMeth': '5',
 'I-GeoMeth': '6',
 'B-GeoPetro': '7',
 'I-GeoPetro': '8'}

In [26]:
#Define the source of model (GeoBERT) for Transfer Learning by loading the previous Weights
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
#     from_pt=True
)

All model checkpoint layers were used when initializing TFBertForTokenClassification.

All the layers of TFBertForTokenClassification were initialized from the model checkpoint at botryan96/GeoBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [27]:
model.get_output_embeddings

<bound method TFPreTrainedModel.get_output_embeddings of <transformers.models.bert.modeling_tf_bert.TFBertForTokenClassification object at 0x00000289BA3F9070>>

In [28]:
#Define Training Optimization Fine-Tuning Hyperparameter
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 5
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [32]:
#Conduct the Training
model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    epochs=num_epochs,
)

Epoch 1/5
2800/2800 [==============================] - 3004s 1s/step - loss: 0.0278 - val_loss: 0.0388
Epoch 2/5
2800/2800 [==============================] - 3001s 1s/step - loss: 0.0139 - val_loss: 0.0351
Epoch 3/5
2800/2800 [==============================] - 3004s 1s/step - loss: 0.0074 - val_loss: 0.0373
Epoch 4/5
2800/2800 [==============================] - 3004s 1s/step - loss: 0.0040 - val_loss: 0.0376
Epoch 5/5
2800/2800 [==============================] - 2993s 1s/step - loss: 0.0030 - val_loss: 0.0376


# G. Push the Model into the Huggingface_Hub

In [2]:
# repo = Repository("GeoBERT", clone_from="botryan96/GeoBERT")
# model.save_pretrained(repo.local_dir)
# tokenizer.save_pretrained(repo.local_dir)
# repo.git_add()
# # repo.git_commit("2nd Trial Version of GeoBERT")
# repo.git_pull()
# repo.git_push()

In [ ]:
import time
time.sleep(300)

# E. Evaluation Matrix

In [36]:
import evaluate

metric = evaluate.load("seqeval")

In [37]:
label_names = list(label2id.keys())

In [38]:
import numpy as np

all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(label_names[predicted_idx])
            all_labels.append(label_names[label_idx])
metric.compute(predictions=[all_predictions], references=[all_labels])

C:\Users\Admin\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:716: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


1/1 [==============================] - 0s 216ms/step


1/1 [==============================] - 0s 191ms/step


1/1 [==============================] - 0s 189ms/step


1/1 [==============================] - 0s 187ms/step


{'GeoLoc': {'precision': 0.8992805755395683,
  'recall': 0.8815232722143864,
  'f1': 0.8903133903133903,
  'number': 709},
 'GeoMeth': {'precision': 0.942671714440238,
  'recall': 0.9601542416452442,
  'f1': 0.9513326662421541,
  'number': 5446},
 'GeoPetro': {'precision': 0.9648360885402073,
  'recall': 0.9521636941794552,
  'f1': 0.9584580057059355,
  'number': 7233},
 'GeoTime': {'precision': 0.9738372093023255,
  'recall': 0.9410112359550562,
  'f1': 0.9571428571428571,
  'number': 356},
 'overall_precision': 0.9527834450597493,
 'overall_recall': 0.9513969732246799,
 'overall_f1': 0.9520897043832824,
 'overall_accuracy': 0.9929089500739384}

# F. Auto Restart for Training

In [ ]:
from IPython.display import HTML, Javascript, display

def initialize():
    display(HTML(
        '''
            <script>
                code_show = false;
                function restart_run_all(){
                    IPython.notebook.kernel.restart();
                    setTimeout(function(){
                        IPython.notebook.execute_all_cells();
                    }, 10000)
                }
                function code_toggle() {
                    if (code_show) {
                        $('div.input').hide(200);
                    } else {
                        $('div.input').show(200);
                    }
                    code_show = !code_show
                }
                restart_run_all();
            </script>
        '''
    ))
initialize()